<a href="https://colab.research.google.com/github/HEM2058/sentinelhub_remote_sensing/blob/main/conversion_yield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install necessary libraries
!pip install geopandas shapely pyproj

import geopandas as gpd
from shapely.geometry import shape

# Step 2: Load the GeoJSON file
geojson_path = 'farm.geojson'  # Update with the path to your file
gdf = gpd.read_file(geojson_path)

# Print the original CRS to ensure it's in EPSG:4326
print(f"Original CRS: {gdf.crs}")

# Step 3: Reproject to a suitable UTM zone for accurate area calculation
# First, find the UTM zone for the centroid of the geometries
centroid = gdf.geometry.centroid.to_crs(epsg=4326).iloc[0]
utm_zone = int((centroid.x + 180) // 6) + 1
utm_crs = f"EPSG:{32600 + utm_zone}"

# Reproject to the UTM CRS
gdf_utm = gdf.to_crs(utm_crs)
print(f"Reprojected to CRS: {gdf_utm.crs}")

# Step 4: Calculate the area of each polygon in acres
# Convert square meters to acres (1 square meter = 0.000247105 acres)
gdf_utm['area_acres'] = gdf_utm['geometry'].area * 0.000247105

# Step 5: Calculate yield per acre
gdf_utm['yield_in_kg_per_acre'] = gdf_utm['yield_in_kg'] / gdf_utm['area_acres']

# Step 6: Reproject back to the original CRS (EPSG:4326)
gdf_final = gdf_utm.to_crs(epsg=4326)

# Step 7: Save the updated GeoJSON
updated_geojson_path = 'farm_yield.geojson'
gdf_final.to_file(updated_geojson_path, driver='GeoJSON')

# Output the first few rows to verify
gdf_final.head()


Original CRS: EPSG:4979
Reprojected to CRS: EPSG:32645


<ipython-input-1-4dc9fc62b8a3>:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid.to_crs(epsg=4326).iloc[0]


,name,yield_in_kg,geometry,area_acres,yield_in_kg_per_acre
0,dada1,240.0,"POLYGON ((86.28216 26.62460, 86.28205 26.62439...",0.219455,1093.620157
1,dada2,230.0,"POLYGON ((86.28178 26.62472, 86.28179 26.62469...",0.239764,959.274778
2,dada3,240.0,"POLYGON ((86.28186 26.62495, 86.28185 26.62492...",0.260580,921.021924
3,Govind_Field1,NaN,"POLYGON ((86.82686 26.56977, 86.82684 26.56951...",1.961400,NaN
4,Govind_Field2,NaN,"POLYGON ((86.82701 26.57143, 86.82700 26.57133...",0.664997,NaN
